# CDF Evaluation

Evaluate the LTSS risk CDFM using limits of agreement, and LoS stratification (separated by "long"/"short" as well as by total length of stay).

In [ ]:
import os
import pickle
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

import sys
sys.path.append('..')

from ltss import los_model, risk_model, vectorise
from ltss.utils import reshape_vector, vector_to_dict
from training.loader import DataHandler

## Generate results using LOS and CDF Models

In [ ]:
LOS_MODEL = los_model.init_model(model_file='../training/mod_ep_110')
CDF_MODEL = risk_model.init_model(model_file='../training/trained_shuffle_100_v2.pkl')
if os.path.exists('checkpoint.pkl'):
    print('Loading Checkpoint')
    with open('checkpoint.pkl', 'rb') as f:
        d = pickle.load(f)
    print(d.keys(), 'containing', len(d['dataset']), 'entries')
    dataset = d['dataset']
    true_los = d['true_los']
    results = d['results']
else:
    # Get a DataHandler with the same shuffle and seed settings as used in training
    loader = DataHandler('../NHSX Polygeist data 1617 to 2021 v2.csv', shuffle=True, fixed_seed=100, reshape=False)
    print(f'Loaded {loader}')
    # Use the validation cut of the data only
    data, true_los = loader.get_validation()
    true_los = true_los.cpu().detach().numpy()
    # Flush streams to sync before using tqdm again
    sys.stdout.flush()
    sys.stderr.flush()
    # Get classification for validation data through both models at once
    results = []
    for i, vector in enumerate(tqdm(data)):
        # Convert to dict representation for the risk model, and predict LTSS
        vector_dict = vector_to_dict(vector)
        forecast = los_model.get_prediction(LOS_MODEL, vector_dict)
        risk_predictions = risk_model.get_prediction(CDF_MODEL, vector_dict, 
                                                     ai_day_prediction=forecast.get('PREDICTED_LOS'))
        # Store results
        results.append(dict(forecast, **risk_predictions))
    dataset = data.cpu().detach().numpy()
    # Save checkpoint for fast reuse
    with open('checkpoint.pkl', 'wb') as f:
        pickle.dump(dict(dataset=dataset, true_los=true_los, results=results), f)

In [ ]:
computed_risk_score = np.asarray([results[i]['RISK_STRATIFICATION'] for i in range(len(results))])
true_risk_score = np.asarray([CDF_MODEL.risk_from_day(x) for x in true_los])

print(f'Generated {len(computed_risk_score)} LTSS predictions from {len(true_risk_score)} ground truth entries')

# Agreement between True and Predicted LTSS

In [ ]:
%matplotlib notebook

In [ ]:
def evaluate(true_los, predicted_los, title, alpha=0.2, cats=None):
    irr_x = true_los
    irr_y = true_los - predicted_los
    mean = np.mean(irr_y)
    stdev = np.std(irr_y)
    limits = [mean + 1.96 * stdev, mean - 1.96 * stdev]
    for confidence in [0.5, 0.75, 0.9, 0.95]:
        sds = stats.norm.ppf(1 - (1 - confidence) / 2)
        print(f'{title} {int(100 * confidence)}% limits of agreement: ±{sds*stdev:0.2f}')
    plt.figure()
    plt.title(title)
    categories = [[] for i in range(np.max(irr_x))]
    for cat, v in zip(irr_x, predicted_los):
        categories[cat - 1].append(v)
    plt.boxplot(categories)
    plt.xlabel('True Risk Score')
    plt.ylabel('Predicted Risk Score')
    plt.show()

In [ ]:
evaluate(true_risk_score, computed_risk_score, 'All Validation Data')

## Stratification of Scores

Separated by "long" and "short" stay, using the 15-day long stay definition discussed in the report.

In [ ]:
def evaluate_stratification(use_los, use_risk_scores, title='Risk Stratification by Stayer Group', long_stay=15):
    long_stayers = np.zeros(5)
    short_stayers = np.zeros(5)
    for i in range(5):
        los_in_band = use_los[use_risk_scores == (i + 1)]
        count_in_band = len(los_in_band)
        if count_in_band > 0:
            long_stayers[i] = np.count_nonzero(los_in_band >= long_stay) / count_in_band
            short_stayers[i] = np.count_nonzero(los_in_band < long_stay) / count_in_band
    print(f'Long stay by band:  {", ".join([f"{v:.2f}" for v in long_stayers])}')
    print(f'Short stay by band: {", ".join([f"{v:.2f}" for v in short_stayers])}')

    labels = ['Risk 1', 'Risk 2', 'Risk 3', 'Risk 4', 'Risk 5']
    width = 0.35  # the width of the bars: can also be len(x) sequence

    fig, ax = plt.subplots()
    ax.bar(labels, short_stayers, width, label=f'Stayers < {long_stay} Days')
    ax.bar(labels, long_stayers, width, bottom=short_stayers, label=f'Long Stayers >= {long_stay} Days')
    ax.set_ylabel('Probability of Score')
    ax.set_title(title)
    ax.legend(loc='lower left')
    plt.show()

In [ ]:
evaluate_stratification(true_los, computed_risk_score, 'All Validation Data')

## Stratification by banded Length of Stay

As above, but instead of separating using "long" and "short" stay, instead show bands of duration.

Plot these with two different spands of duration band, in order to show there isn't an "end of week" peak effect skewing the results.

In [ ]:
# Arbitrary maximum length of stay (in practice our vectorised LoS estimates are clamped to max 30 days)
MAX = 100

def evaluate_stratification_banded(use_los, use_risk_scores, title='Risk Stratification by Stayer Group', 
                                   stays = [7, 14, 21, MAX]):
    stayer_bands = [np.zeros(5) for _ in stays]
    for i in range(5):
        los_in_band = use_los[use_risk_scores == (i + 1)]
        count_in_band = len(los_in_band)
        if count_in_band > 0:
            prev_stay = 0
            for los, band in zip(stays, stayer_bands):
                curr_proportion = np.count_nonzero(los_in_band < los) / count_in_band
                band[i] = curr_proportion - prev_stay
                prev_stay = curr_proportion

    labels = ['Risk 1', 'Risk 2', 'Risk 3', 'Risk 4', 'Risk 5']
    width = 0.35  # the width of the bars: can also be len(x) sequence
    print('\n'.join([f'{l}: ' + ', '.join([f'{v:.2f}' for v in a]) for l, a in zip(stays, stayer_bands)]))

    fig, ax = plt.subplots()
    prev_band = None
    prev_los = None
    for los, band in zip(stays, stayer_bands):
        if prev_band is None:
            ax.bar(labels, band, width, label=f'Stay < {los} Days')
            prev_band = band
        else:
            ax.bar(labels, band, width, bottom=prev_band, 
                   label=f'{prev_los} <= Stay < {los} Days' if los < MAX else f'Stay >= {prev_los} Days')
            prev_band += band
        prev_los = los
    ax.set_ylabel('Probability of Length of Stay in Band')
    ax.set_title(title)
    ax.legend(loc='lower left')
    plt.show()

In [ ]:
evaluate_stratification_banded(true_los, computed_risk_score, 'All Validation Data')
evaluate_stratification_banded(true_los, computed_risk_score, 'All Validation Data', stays=[5, 10, 15, 20, 25, MAX])